<a href="https://colab.research.google.com/github/lozrigby/lab-chatbot/blob/main/lab_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [ ]:
pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os



In [ ]:
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file

# OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('Key_from_ironhack')

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [ ]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, of course! Such a classic jest that doth never fail to amuse.


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you today?


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't actually know your name. How about you tell me what it is and I can remember it for our future conversations?


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "cheese pizza",
    "size": "medium"
  },
  "toppings": [],
  "drinks": [],
  "sides": [],
  "total price": 10.95
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
text = f"""
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""
prompt = f"""
Select a pizza, topping and drink to create the most expensive menu option.
```{text}```
"""
response = get_completion(prompt)
print(response)

The most expensive menu option would be an Eggplant Pizza with extra cheese topping and a bottled water, totaling $19.95.


In [ ]:
text = f"""
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""
prompt = f"""
Select the most expensive large pizza, the most expensive topping and most expensive drink to create the most expensive menu option.
```{text}```
"""
response = get_completion(prompt)
print(response)

The most expensive large pizza is the pepperoni pizza at $12.95. 
The most expensive topping is sausage at $3.00. 
The most expensive drink is bottled water at $5.00. 

Therefore, the most expensive menu option would be a pepperoni pizza with sausage topping and bottled water, totaling $20.95.


Experiment with sequences

In [ ]:
text_1 = f"""
In order to have the best sleep possible, you firstly need \
to be in a sleepy mood, next you should go and brush your teeth \
before brushing you need to put your pjs on. \
After you have brushed your teeth you should wash your face . \
Then if you want to you can read a book or listen to calming music \
When you are peak sleepy turn off your lights and go to sleep .
Between washing your face and reading it is helpful to floss.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Be in a sleepy mood.
Step 2 - Put on your pajamas.
Step 3 - Brush your teeth.
Step 4 - Wash your face.
Step 5 - Floss (optional).
Step 6 - Read a book or listen to calming music.
Step 7 - Turn off the lights and go to sleep.


In [ ]:
prompt = f"""
Tell me how to make the most money betting
"""
response = get_completion(prompt)
print(response)

There is no guaranteed way to make money betting as it involves a certain level of risk. However, here are some tips to potentially increase your chances of making money:

1. Do your research: Before placing any bets, make sure to research the teams or players involved, their recent performance, injuries, and any other relevant information that could affect the outcome of the event.

2. Manage your bankroll: Set a budget for your betting activities and stick to it. Avoid chasing losses by betting more than you can afford to lose.

3. Shop for the best odds: Different bookmakers offer different odds for the same event, so make sure to compare odds and choose the best value for your bets.

4. Consider different betting strategies: There are various betting strategies such as value betting, arbitrage betting, and matched betting that can help increase your chances of making a profit.

5. Stay disciplined: Avoid emotional betting and stick to your strategy. Don't let a few losses discourag

In [ ]:
prompt = f"""
Tell me about the artist Loz Rigby
"""
response = get_completion(prompt)
print(response)

Loz Rigby is a contemporary artist based in the United Kingdom. She is known for her vibrant and expressive abstract paintings that often feature bold colors and dynamic compositions. Rigby's work is influenced by a variety of sources, including nature, music, and emotions.

Rigby's artistic process involves layering paint and other materials to create rich textures and depth in her pieces. She often incorporates elements of collage and mixed media into her work, adding further complexity and visual interest.

Rigby has exhibited her work in galleries and art fairs both in the UK and internationally. She has gained a following for her unique and energetic style, which has been described as both playful and sophisticated.

Overall, Loz Rigby is a talented and innovative artist who continues to push the boundaries of abstract art with her dynamic and engaging creations.
